In [28]:
import pandas as pd
import numpy as np

In [29]:
file_path = '/Users/diana/Desktop/isear-test.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [30]:
df.head ()

,Emotions,Text
0,joy,My first feeling of falling in love. Coming a...
1,joy,When I saw that the mark I had obtained in an ...
2,joy,When I learnt that I had been admitted to the ...
3,anger,In a hurry to get to lectures and the car woul...
4,guilt,"Not attending school, when am not sick - but ..."


In [31]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

1150


In [32]:
df['Emotions'].value_counts()

Emotions
shame      182
fear       168
anger      166
disgust    166
sadness    161
guilt      155
joy        152
Name: count, dtype: int64

In [33]:
emotion_labels = list (set (df['Emotions'].values))

In [56]:
#print (df['Text'].dtype)
df['Text'] = df['Text'].str.lower ()
#print (df['Text'])

text_column = df['Text']
tokenized_text = text_column.str.split()

for index, sentence in enumerate(tokenized_text):
    emotion_label = df['Emotions'][index]
    #print(emotion_label, sentence)

def tokenize(text):
    translation_table = str.maketrans({c: f' {c} ' if not c.isalnum() else c for c in set(text)})  # creates translation
    # table(dictionary) with the built-in function maketrans, set(text) makes an unordered collection of unique
    # elements through set comprehension, a concise way to create sets
    tokenized_text = text.translate(translation_table)  # uses translation table to add whitespace around special
    # characters and punctuation
    return tokenized_text.strip().lower().split()  # split-tokens split on space, lower-tokens made all lowercase,
    # strip-and leading or trailing whitespaces are removed from string
print (len(tokenized_text))

#extract dict
text = ''.join(df['Text'].astype(str))
corpus = tokenize(text) #all tokens, including repeating
vocab = set(corpus)
#print(vocab)


1150


In [57]:
from math import log

collection = [token for instance in tokenized_text for token in instance]
#print (collection)

#tf-idf

def calculate_tfidf(token, document):
    tf = document.count(token) / len(document) # 1+log(#oftimestokenindoc/total#termsindoc)
    idf = log (1 + (len(tokenized_text) / collection.count(token))) # log(#ofdocs/#oftimestokenappearsincollection)
    tfidf = tf * idf
    return tfidf

for instance in tokenized_text:
        tfidf_scores = {token: calculate_tfidf(token, instance) for token in set(instance)}
        print (tfidf_scores)

{'coming': 0.1576893547456502, 'love,': 0.2191761262308928, 'feel': 0.12662318399301914, 'person': 0.11536959411523884, 'joy,': 0.2430478071973063, 'first': 0.11382326413911331, 'across': 0.2052244907448603, 'me': 0.17720468186637317, 'love.': 0.18766957622488942, 'in': 0.04435726736074292, 'of': 0.043198723051564414, 'happiness,': 0.2430478071973063, 'and': 0.03498808100570242, 'made': 0.22051192116668192, 'feeling': 0.13708360402684971, 'falling': 0.19533431330855705, 'dream.': 0.2430478071973063, 'beautiful': 0.18141246827600924, 'giving': 0.2052244907448603, 'who': 0.186094130715017, 'a': 0.057601788665117594, 'my': 0.03458972029727068, 'future,': 0.2430478071973063, 'new': 0.17155200401562432}
{'in': 0.08039754709134654, 'an': 0.14470022230957857, 'obtained': 0.3972567287934932, 'good': 0.2201343745049886, 'exam': 0.25746656139410173, 'i': 0.06879567088379086, 'the': 0.05133342897512176, 'was': 0.06485152629677803, 'saw': 0.2063046662521429, 'mark': 0.32881009875026673, 'one.': 0.

In [ ]:
#conda install scikit-learn

In [ ]:
#import sklearn

In [ ]:
"""from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus to obtain the TF-IDF sparse matrix
tfidf_matrix_sparse = tfidf_vectorizer.fit_transform(sentence)

#tfidf_vector_first_sentence = tfidf_matrix_sparse[0]
#print("First", tfidf_vector_first_sentence)
print (tfidf_matrix_sparse)"""



'from sklearn.feature_extraction.text import TfidfVectorizer\n\n\ntfidf_vectorizer = TfidfVectorizer()\n\n# Fit and transform the corpus to obtain the TF-IDF sparse matrix\ntfidf_matrix_sparse = tfidf_vectorizer.fit_transform(sentence)\n\n#tfidf_vector_first_sentence = tfidf_matrix_sparse[0]\n#print("First", tfidf_vector_first_sentence)\nprint (tfidf_matrix_sparse)'

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.autograd import Variable

In [ ]:
class Emotions (nn.Module):

    def __init__(self, input_size, hidden_1, output_size):

        super (Emotions, self).__init__()
        self.fc1 = nn.Linear (input_size, hidden_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear (hidden_1, output_size)
    def forward (self, x):
        out = self.fc1 (x)
        out = self.relu1 (out)
        out = self.fc2 (out)
        return out



In [ ]:
model = Emotions (1150, 50, 7)

In [ ]:
import torch.utils

batch_size = 50
train_loader = torch.utils.data.DataLoader (dataset = df, batch_size=batch_size, shuffle =True)